# TV Script Generation
In this project, you'll generate your own [Simpsons](https://en.wikipedia.org/wiki/The_Simpsons) TV scripts using RNNs.  You'll be using part of the [Simpsons dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data) of scripts from 27 seasons.  The Neural Network you'll build will generate a new TV script for a scene at [Moe's Tavern](https://simpsonswiki.com/wiki/Moe's_Tavern).
## Get the Data
The data is already provided for you.  You'll be using a subset of the original dataset.  It consists of only the scenes in Moe's Tavern.  This doesn't include other versions of the tavern, like "Moe's Cavern", "Flaming Moe's", "Uncle Moe's Family Feed-Bag", etc..

In [2]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper

data_dir = './data/simpsons/moes_tavern_lines.txt'
text = helper.load_data(data_dir)
# Ignore notice, since we don't use it for analysing the data
text = text[81:]

## Explore the Data
Play around with `view_sentence_range` to view different parts of the data.

In [3]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 11492
Number of scenes: 262
Average number of sentences in each scene: 15.248091603053435
Number of lines: 4257
Average number of words in each line: 11.50434578341555

The sentences 0 to 10:
Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.
Bart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.
Moe_Szyslak: (INTO PHONE) Hold on, I'll check. (TO BARFLIES) Mike Rotch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?
Moe_Szyslak: (INTO PHONE) Listen you little puke. One of these days I'm gonna catch you, and I'm gonna carve my name on your back with an ice pick.
Moe_Szyslak: What's the matter Homer? You're not your normal effervescent self.
Homer_Simpson: I got my problems, Moe. Give me another one.
Moe_Szyslak: Homer, hey, you should not drink to forget your problems.
Barney_Gumble: Yeah, you should only drink to enhance your social skills.




## Implement Preprocessing Functions
The first thing to do to any dataset is preprocessing.  Implement the following preprocessing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following tuple `(vocab_to_int, int_to_vocab)`

In [4]:
import numpy as np
import problem_unittests as tests
from collections import Counter 

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    # TODO: Implement Function
    vocab = Counter(text)
    
    vocab_to_int = { word: nr for nr, word in enumerate(vocab) }
    int_to_vocab = { nr: word for word, nr in vocab_to_int.items() }
    
    return vocab_to_int, int_to_vocab


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks make it hard for the neural network to distinguish between the word "bye" and "bye!".

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( . )
- Comma ( , )
- Quotation Mark ( " )
- Semicolon ( ; )
- Exclamation mark ( ! )
- Question mark ( ? )
- Left Parentheses ( ( )
- Right Parentheses ( ) )
- Dash ( -- )
- Return ( \n )

This dictionary will be used to token the symbols and add the delimiter (space) around it.  This separates the symbols as it's own word, making it easier for the neural network to predict on the next word. Make sure you don't use a token that could be confused as a word. Instead of using the token "dash", try using something like "||dash||".

In [5]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenize dictionary where the key is the punctuation and the value is the token
    """
    # TODO: Implement Function
    return {
        '.': "||Period||",
        ',': "||Comma||",
        '"' : "||Quotation_Mark||",
        ';' : "||Semicolon||",
        '!' : "||Exclamation_mark||",
        '?' : "||Question_mark||",
        '(' : "||Left_Parentheses||",
        ')' : "||Right_Parentheses||",
        '--' : "||Dash||",
        '\n' : "||Return||"
    }

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

Tests Passed


## Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Preprocess Training, Validation, and Testing Data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import numpy as np
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Build the Neural Network
You'll build the components necessary to build a RNN by implementing the following functions below:
- get_inputs
- get_init_cell
- get_embed
- build_rnn
- build_nn
- get_batches

### Check the Version of TensorFlow and Access to GPU

In [2]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.0.1
Default GPU Device: /gpu:0


### Input
Implement the `get_inputs()` function to create TF Placeholders for the Neural Network.  It should create the following placeholders:
- Input text placeholder named "input" using the [TF Placeholder](https://www.tensorflow.org/api_docs/python/tf/placeholder) `name` parameter.
- Targets placeholder
- Learning Rate placeholder

Return the placeholders in the following the tuple `(Input, Targets, LearingRate)`

In [3]:
def get_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate)
    """
    # TODO: Implement Function
    inputs = tf.placeholder(tf.int32, [None, None], name="input")
    targets = tf.placeholder(tf.int32, [None, None], name="target")
    
    learning_rate = tf.placeholder(tf.float32, name="learning_rate")
    
    return inputs, targets, learning_rate


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_inputs(get_inputs)

Tests Passed


### Build RNN Cell and Initialize
Stack one or more [`BasicLSTMCells`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/BasicLSTMCell) in a [`MultiRNNCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell).
- The Rnn size should be set using `rnn_size`
- Initalize Cell State using the MultiRNNCell's [`zero_state()`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell#zero_state) function
    - Apply the name "initial_state" to the initial state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the cell and initial state in the following tuple `(Cell, InitialState)`

In [4]:
def get_init_cell(batch_size, rnn_size):
    """
    Create an RNN Cell and initialize it.
    :param batch_size: Size of batches
    :param rnn_size: Size of RNNs
    :return: Tuple (cell, initialize state)
    """
    # TODO: Implement Function

    with tf.name_scope("lstm_layers"):
        cell = tf.contrib.rnn.BasicLSTMCell(rnn_size)
        cell = tf.contrib.rnn.MultiRNNCell([cell] * 2)

    with tf.name_scope("lstm_initial_state"):
        initialState = cell.zero_state(batch_size, tf.float32)

    initialState = tf.identity(initialState, "initial_state")

    return cell, initialState


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_init_cell(get_init_cell)

Tests Passed


### Word Embedding
Apply embedding to `input_data` using TensorFlow.  Return the embedded sequence.

In [5]:
def get_embed(input_data, vocab_size, embed_dim):
    """
    Create embedding for <input_data>.
    :param input_data: TF placeholder for text input.
    :param vocab_size: Number of words in vocabulary.
    :param embed_dim: Number of embedding dimensions
    :return: Embedded input.
    """
    # TODO: Implement Function
    with tf.name_scope("word_embedding"):
        embedding = tf.Variable(tf.random_uniform((vocab_size, embed_dim), -1, 1))
        embed = tf.nn.embedding_lookup(embedding, input_data)
    return embed


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_embed(get_embed)

Tests Passed


### Build RNN
You created a RNN Cell in the `get_init_cell()` function.  Time to use the cell to create a RNN.
- Build the RNN using the [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)
 - Apply the name "final_state" to the final state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the outputs and final_state state in the following tuple `(Outputs, FinalState)` 

In [6]:
def build_rnn(cell, inputs):
    """
    Create a RNN using a RNN Cell
    :param cell: RNN Cell
    :param inputs: Input text data
    :return: Tuple (Outputs, Final State)
    """
    # TODO: Implement Function
    
    rnn, final_state = tf.nn.dynamic_rnn(cell, inputs, dtype=tf.float32)
    final_state = tf.identity(final_state, "final_state")
    return rnn, final_state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_rnn(build_rnn)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:
- Apply embedding to `input_data` using your `get_embed(input_data, vocab_size, embed_dim)` function.
- Build RNN using `cell` and your `build_rnn(cell, inputs)` function.
- Apply a fully connected layer with a linear activation and `vocab_size` as the number of outputs.

Return the logits and final state in the following tuple (Logits, FinalState) 

In [7]:
def build_nn(cell, rnn_size, input_data, vocab_size):
    """
    Build part of the neural network
    :param cell: RNN cell
    :param rnn_size: Size of rnns
    :param input_data: Input data
    :param vocab_size: Vocabulary size
    :return: Tuple (Logits, FinalState)
    """
    # TODO: Implement Function
    
    embedding = get_embed(input_data, vocab_size, embed_dim=200)
    output, final_state = build_rnn(cell, inputs=embedding)
    fully_con = tf.contrib.layers.fully_connected(inputs=output, num_outputs=vocab_size, activation_fn=None)
    
    return fully_con, final_state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_nn(build_nn)

Tests Passed


### Batches
Implement `get_batches` to create batches of input and targets using `int_text`.  The batches should be a Numpy array with the shape `(number of batches, 2, batch size, sequence length)`. Each batch contains two elements:
- The first element is a single batch of **input** with the shape `[batch size, sequence length]`
- The second element is a single batch of **targets** with the shape `[batch size, sequence length]`

If you can't fill the last batch with enough data, drop the last batch.

For exmple, `get_batches([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15], 2, 3)` would return a Numpy array of the following:
```
[
  # First Batch
  [
    # Batch of Input
    [[ 1  2  3], [ 7  8  9]],
    # Batch of targets
    [[ 2  3  4], [ 8  9 10]]
  ],
 
  # Second Batch
  [
    # Batch of Input
    [[ 4  5  6], [10 11 12]],
    # Batch of targets
    [[ 5  6  7], [11 12 13]]
  ]
]
```

In [8]:
def get_batches(int_text, batch_size, seq_length):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: Batches as a Numpy array
    """
    # TODO: Implement Function
    n_batches = int(len(int_text) / (batch_size * seq_length))

    # Drop the last few characters to make only full batches
    xdata = np.array(int_text[: n_batches * batch_size * seq_length])
    ydata = np.array(int_text[1: n_batches * batch_size * seq_length + 1])

    x_batches = np.split(xdata.reshape(batch_size, -1), n_batches, 1)
    y_batches = np.split(ydata.reshape(batch_size, -1), n_batches, 1)
    
    #print(x_batches)
    batches = np.array([x_batches, y_batches])
    
    return np.rot90(batches)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_batches(get_batches)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `num_epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `seq_length` to the length of sequence.
- Set `learning_rate` to the learning rate.
- Set `show_every_n_batches` to the number of batches the neural network should print progress.

In [9]:
# Number of Epochs
num_epochs = [1]
# Batch Size
batch_size = [64, 128, 256]
# RNN Size
rnn_size = 256
# Sequence Length
seq_length = [50, 100, 150]
# Learning Rate
learning_rate = [0.005, 0.001, 0.0005]
# Show stats for every n number of batches
show_every_n_batches = 1

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
#save_dir = './save'

"\nDON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE\n"

### Build the Graph
Build the graph using the neural network you implemented.

In [10]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    
    with tf.name_scope("inputs"):
        input_text, targets, lr = get_inputs()
        input_data_shape = tf.shape(input_text)
        
    with tf.name_scope("lstm"):
        cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    
    with tf.name_scope("rnn_nn"):
        logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size)

    # Probabilities for generating words
    
    with tf.name_scope("activation"):
        probs = tf.nn.softmax(logits, name='probs')
        tf.summary.histogram('predictions', probs)
        
    with tf.name_scope("back_propagation"):
        # Loss function
        cost = seq2seq.sequence_loss(
            logits,
            targets,
            tf.ones([input_data_shape[0], input_data_shape[1]]))
        
        tf.summary.scalar('cost', cost)

        # Optimizer
        optimizer = tf.train.AdamOptimizer(lr)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients]
        train_op = optimizer.apply_gradients(capped_gradients)
    
    merged = tf.summary.merge_all()

## Train
Train the neural network on the preprocessed data.  If you have a hard time getting a good loss, check the [forms](https://discussions.udacity.com/) to see if anyone is having the same problem.

In [13]:
batches = get_batches(int_text, batch_size[0], seq_length[0])
for batch_i, (x, y) in enumerate(batches):
    print(x, y)

[[5018 1171 2047 ..., 4500 1107 3950]
 [1037 1125 1614 ..., 1833  215 3898]
 [2183 1614 5559 ...,  215 1735 2844]
 ..., 
 [ 215 3898  511 ..., 4924 5798 5733]
 [3422 2963 2465 ...,  319 2963 2712]
 [6545 4054 3493 ..., 3976 4652 5637]] [[1171 2047 3515 ..., 1107 3950 2157]
 [1125 1614 5324 ...,  215 3898 5815]
 [1614 5559  215 ..., 1735 2844 2193]
 ..., 
 [3898  511 2171 ..., 5798 5733  215]
 [2963 2465  215 ..., 2963 2712 4494]
 [4054 3493 1901 ..., 4652 5637 5761]]
[[6149 4039 3898 ...,  690 2995 3950]
 [4322 2899 5817 ..., 2465 4243 3466]
 [4243  712 4039 ...,  215  855 3468]
 ..., 
 [3405 1614 6683 ..., 5015 3045 2339]
 [4162 1776 4587 ..., 4039 3898 1602]
 [3566 4746  686 ..., 5552 2963 4243]] [[4039 3898 5815 ..., 2995 3950 5018]
 [2899 5817 4924 ..., 4243 3466 1037]
 [ 712 4039 3898 ...,  855 3468 2183]
 ..., 
 [1614 6683 3012 ..., 3045 2339  215]
 [1776 4587 2963 ..., 3898 1602 3422]
 [4746  686 1246 ..., 2963 4243 6545]]
[[2891 3233 4039 ...,  829 2235 3688]
 [4134  108 5315 .

In [17]:
!rm -r tensorboard/*
!rm -r trained_models/*

In [18]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import os

def train(num_epochs, batch_size, seq_length, learning_rate):
    batches = get_batches(int_text, batch_size, seq_length)
    
    config = tf.ConfigProto()
    config.gpu_options.per_process_gpu_memory_fraction = 0.4

    with tf.Session(graph=train_graph, config=config) as sess:
        sess.run(tf.global_variables_initializer())

        file_writer = tf.summary.FileWriter("./tensorboard/lr={},bs={},sl={}".format(learning_rate, batch_size, seq_length), sess.graph)
        iteration = 0

        for epoch_i in range(num_epochs):
            state = sess.run(initial_state, {input_text: batches[0][0]})

            for batch_i, (x, y) in enumerate(batches):
                feed = {
                    input_text: x,
                    targets: y,
                    initial_state: state,
                    lr: learning_rate}
                summary, train_loss, state, _ = sess.run([merged, cost, final_state, train_op], feed)

                # Show every <show_every_n_batches> batches
                if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                    print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                        epoch_i,
                        batch_i,
                        len(batches),
                        train_loss))

                file_writer.add_summary(summary, iteration)
                iteration += 1

        # Save Model
        folder = "./trained_models/lr={},bs={},sl={}/model".format(learning_rate, batch_size, seq_length)
        if not os.path.exists(folder):
            os.makedirs(folder)
        saver = tf.train.Saver()
        saver.save(sess, folder)

for num__epochs in num_epochs:
    for batch__size in batch_size:
        for seq__length in seq_length:
            for learning__rate in learning_rate:
                print(num__epochs, batch__size, seq__length, learning__rate)
                train(num__epochs, batch__size, seq__length, learning__rate)
        
print('All Model Trained and Saved')

1 64 50 0.005
Epoch   0 Batch    0/21   train_loss = 8.821
Epoch   0 Batch    1/21   train_loss = 8.727
Epoch   0 Batch    2/21   train_loss = 7.924
Epoch   0 Batch    3/21   train_loss = 7.116
Epoch   0 Batch    4/21   train_loss = 6.566
Epoch   0 Batch    5/21   train_loss = 6.349
Epoch   0 Batch    6/21   train_loss = 6.242
Epoch   0 Batch    7/21   train_loss = 6.384
Epoch   0 Batch    8/21   train_loss = 6.682
Epoch   0 Batch    9/21   train_loss = 6.609
Epoch   0 Batch   10/21   train_loss = 6.607
Epoch   0 Batch   11/21   train_loss = 6.546
Epoch   0 Batch   12/21   train_loss = 6.477
Epoch   0 Batch   13/21   train_loss = 6.524
Epoch   0 Batch   14/21   train_loss = 6.703
Epoch   0 Batch   15/21   train_loss = 6.567
Epoch   0 Batch   16/21   train_loss = 6.584
Epoch   0 Batch   17/21   train_loss = 6.604
Epoch   0 Batch   18/21   train_loss = 6.503
Epoch   0 Batch   19/21   train_loss = 6.495
Epoch   0 Batch   20/21   train_loss = 6.512
1 64 50 0.001
Epoch   0 Batch    0/21   t

ResourceExhaustedError: OOM when allocating tensor with shape[38400,6779]
	 [[Node: activation/probs = Softmax[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/gpu:0"](activation/Reshape)]]
	 [[Node: activation/Reshape_1/_17 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_1365_activation/Reshape_1", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'activation/probs', defined at:
  File "/home/benjamin/anaconda3/envs/dl/lib/python3.5/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/benjamin/anaconda3/envs/dl/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/benjamin/anaconda3/envs/dl/lib/python3.5/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/benjamin/anaconda3/envs/dl/lib/python3.5